In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
import sys
import preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
    # Adjust file paths as needed for your local file system
    file_path = './train_32x32.mat'

    # Check if file is in your specified local directory
    if not os.path.exists(file_path):
        print(file_path," not found, exiting")
        sys.exit()
    else:
        print(file_path, "found")

    # Load the training data
    train_mat = scipy.io.loadmat(file_path)

    # categorize the data
    X_tr = train_mat['X']
    y_tr = train_mat['y']
    y_tr = y_tr.astype(int)

    file_path = "./test_32x32.mat"
    if not os.path.exists(file_path):
        print(file_path," not found, exiting")
        sys.exit()
    else:
        print(file_path, "found")

    # Load the test data
    test_mat = scipy.io.loadmat(file_path)

    X_te = test_mat['X']
    y_te = test_mat['y']
    y_te = y_te.astype(int)
    
    # split = 20000
    # X_tr = X_tr[:, :, :,:split]
    # y_tr = y_tr[:split]
    
    print(X_tr.shape)
    print(y_tr.shape)
    for i in range(y_tr.shape[0]):
        if y_tr[i] == 10:
            y_tr[i] = 0
    for i in range(y_te.shape[0]):
        if y_te[i] == 10:
            y_te[i] = 0

./train_32x32.mat found
./test_32x32.mat found
(32, 32, 3, 73257)
(73257, 1)


In [4]:
    X_tr_reshaped = X_tr.reshape(X_tr.shape[3], -1)
    X_te_reshaped = X_te.reshape(X_te.shape[3], -1)
    
    scaler = MinMaxScaler()

    # Fitting on training, transform on both training and testing
    X_tr_scaled = scaler.fit_transform(X_tr_reshaped)
    X_te_scaled = scaler.transform(X_te_reshaped)

    X_tr = X_tr_scaled.reshape(X_tr.shape)
    X_te = X_te_scaled.reshape(X_te.shape)

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 3 color channels, 6 feature maps to output, and a 5x5 kernel. Stride is 1 for the kernel.
        self.pool = nn.MaxPool2d(2, 2) # Pooling filter is 2x2, and a stride of 2.
        self.conv2 = nn.Conv2d(6, 20, 5) # 6 input feature maps from conv1 + pool, 16 feature maps to output, 5x5 kernel. Stride of 1.
        self.fc1 = nn.Linear(20*5*5, 120) # 16 for the output layer * the resulting 5x5 image after pooling
        self.fc2 = nn.Linear(120, 84) # 120 input layer, 84 output
        self.fc3 = nn.Linear(84, 10) # 84 input layer, 10 output (10 classes)

        # Formula applied for conv = (Dimension - Filter)/Stride + 1


    def forward(self, x, featureMap):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, featureMap*5*5) # Flattening, because Linear only takes a 1D array
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)    # Don't apply ReLU here, nor softmax. CrossEntropyLoss() applies softmax automatically, and requires raw data.
        return x

In [17]:
num_epochs = 10
batch_size = 5
learning_rate = 0.001

device = torch.device("cuda")
criterion = nn.CrossEntropyLoss()
# First, converting our data into tensors
X_tr_transposed = X_tr.transpose((3, 2, 0, 1))

X_tr_tensor = torch.from_numpy(X_tr_transposed).float().to(device)
y_tr_tensor = torch.from_numpy(y_tr).long().to(device)

# Converting our tensors into a Tensor Dataset
train_dataset = torch.utils.data.TensorDataset(X_tr_tensor, y_tr_tensor)

# Creating a loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training the model
n_total_steps = len(train_loader)
epochs = list(range(5,26))
test_error = []
train_error = []
for e in epochs:
    model = ConvNet().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    print(f"Training with {e} Epochs")
    for epoch in range(e):
        print(f"Epoch {epoch + 1}/{e}...")
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            labels = labels.squeeze()
            # Zeroing gradient
            optimizer.zero_grad()

            # Forward pass
            # (batch_size, 3, 32, 32)
            # (32, 32, 3, batch_size)
            outputs = model.forward(inputs, 20)

            # Applying loss
            loss = criterion(outputs, labels)

            # Backpropogation
            loss.backward()

            # Update our weights
            optimizer.step()

            # print(f"Epoch [{epoch + 1}/{e}], Batch [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")
    test_error.append(1-test(X_te, y_te, model, batch_size, 20))
    train_error.append(1-test(X_tr, y_tr, model, batch_size, 20))
plt.plot(epochs, test_error, label="Test Error")
plt.plot(epochs, train_error, label="Train Error")
plt.xlabel("Feature maps")
plt.ylabel("Accuracy")
plt.title("Accuracy vs. Feature maps")
plt.xticks(epochs)
plt.legend()
plt.show()

Training with 5 Epochs
Epoch 1/5...
Epoch 2/5...
Epoch 3/5...


In [7]:
def test(X_te, y_te, model, batch_size, featureMap):
  # Testing the model
  X_te_transposed = X_te.transpose((3, 2, 0, 1))

  X_te_tensor = torch.from_numpy(X_te_transposed).float().to(device)
  y_te_tensor = torch.from_numpy(y_te).long().to(device)

  # Converting our tensors into a Tensor Dataset
  test_dataset = torch.utils.data.TensorDataset(X_te_tensor, y_te_tensor)

  # Creating a loader
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  print("testing the model...")
  correct_count, all_count = 0, 0
  for images,labels in test_loader:
    for i in range(len(labels)):
      img = images[i].view(1, 3, 32, 32)
      with torch.no_grad():
          logps = model(img, featureMap)

      
      ps = torch.exp(logps)
      probab = list(ps.cpu()[0])
      pred_label = probab.index(max(probab))
      true_label = labels.cpu()[i]
      if(true_label == pred_label):
        correct_count += 1
      all_count += 1

  print("Number Of Images Tested =", all_count)
  print("\nModel Accuracy =", (correct_count/all_count))
  return (correct_count/all_count)

In [ ]:
num_epochs = 10
batch_size = 5
learning_rate = 0.001

device = torch.device("cpu")
criterion = nn.CrossEntropyLoss()
# First, converting our data into tensors
X_tr_transposed = X_tr.transpose((3, 2, 0, 1))

X_tr_tensor = torch.from_numpy(X_tr_transposed).float().to(device)
y_tr_tensor = torch.from_numpy(y_tr).long().to(device)

# Converting our tensors into a Tensor Dataset
train_dataset = torch.utils.data.TensorDataset(X_tr_tensor, y_tr_tensor)

# Creating a loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

n_total_steps = len(train_loader)
model = ConvNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
model.conv2 = nn.Conv2d(6, 5, 5)
model.fc1 = nn.Linear(5*5*5, 120)
for epoch in range(num_epochs):
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        labels = labels.squeeze()
        # Zeroing gradient
        optimizer.zero_grad()

        # Forward pass
        # (batch_size, 3, 32, 32)
        # (32, 32, 3, batch_size)
        outputs = model.forward(inputs, 5)

        # Applying loss
        loss = criterion(outputs, labels)

        # Backpropogation
        loss.backward()

        # Update our weights
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")

In [ ]:
test(X_te, y_te, model, batch_size, 5)

changed epoch from 5 to 10, accuracy went from 64 to 83